In [1]:
import math
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re, collections
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
import enchant 
import pandas as pd
import spacy
import re
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import urllib.request
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
tokenizer = RegexpTokenizer(r'\w+')
from sklearn.metrics.pairwise import euclidean_distances


In [4]:
dataframe = pd.read_csv('training_set_rel3.tsv', encoding = 'latin-1',sep='\t')
dataframe = dataframe[['essay_id','essay_set','essay','domain1_score']]
dataframe.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [5]:
dataframe = dataframe.iloc[:5]
dataframe.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [6]:
dataframe.set_index('essay_id',inplace=True, drop=True)

In [7]:

def get_string_arrays(regExTokenizer, df):
    array_of_strings = []
    for essay in df['essay']:
        words = regExTokenizer.tokenize(essay)
        array = []
        size = len(words)//4
        curr = 0
        flag = 0
        while flag == 0:
            new = ' '
            for i in range(curr,curr+size):
                if i==len(words):
                    flag = 1
                    break
                new = new+' '+words[i]
            curr+=10
            array.append(new)
        array_of_strings.append(array)
    return array_of_strings
array_of_strings = get_string_arrays(tokenizer,dataframe)
print(array_of_strings)
    
        

[['  Dear local newspaper I think effects computers have on people are great learning skills affects because they give us time to chat with friends new people helps us learn about the globe astronomy and keeps us out of troble Thing about Dont you think so How would you feel if your teenager is always on the phone with friends Do you ever time to chat with your friends or buisness partner about things Well now there s a new way to chat the computer theirs plenty of', '  are great learning skills affects because they give us time to chat with friends new people helps us learn about the globe astronomy and keeps us out of troble Thing about Dont you think so How would you feel if your teenager is always on the phone with friends Do you ever time to chat with your friends or buisness partner about things Well now there s a new way to chat the computer theirs plenty of sites on the internet to do so ORGANIZATION1 ORGANIZATION2 CAPS1', '  to chat with friends new people helps us learn about

In [8]:

tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
cosine_array = []
for i in range(0,len(array_of_strings)):
    tfidf_wm = tfidfvectorizer.fit_transform(array_of_strings[i])
    similarity_matrix = cosine_similarity(tfidf_wm)
    cosine_array.append(similarity_matrix)

In [9]:
print(cosine_array)

[array([[1.        , 0.84771825, 0.73215724, 0.61438601, 0.4873051 ,
        0.41026711, 0.36400028, 0.27985365, 0.2480269 , 0.17654184,
        0.18459333, 0.16412095, 0.14635954, 0.13698783, 0.12360375,
        0.14702075, 0.13218949, 0.08749086, 0.0612726 , 0.06309653,
        0.08339863, 0.07447685, 0.07330877, 0.14984849, 0.18512737,
        0.31310039, 0.36636506, 0.38032999],
       [0.84771825, 1.        , 0.86216713, 0.76608957, 0.62764911,
        0.55414205, 0.50641112, 0.42341499, 0.37920256, 0.27374055,
        0.19522781, 0.1853553 , 0.165785  , 0.15543834, 0.14056805,
        0.12508446, 0.10860068, 0.07315636, 0.04506391, 0.04640535,
        0.07059497, 0.06177498, 0.06080612, 0.08346712, 0.14462148,
        0.25794676, 0.31708641, 0.32917296],
       [0.73215724, 0.86216713, 1.        , 0.90937225, 0.77169509,
        0.68428178, 0.63980242, 0.55853222, 0.52019185, 0.40340929,
        0.24445084, 0.1827077 , 0.16249414, 0.15184704, 0.13672608,
        0.12083358, 0.103

In [10]:
len(cosine_array)

5

In [11]:
def get_avg_distance_bw_neighbours(cosine_array):
    array = []
    for i in range(0,len(cosine_array)):
        j = 0
        dist = 0
        while j < cosine_array[i].shape[0]-1:
            dist+=cosine_array[i][j][j+1]
            j+=1
        array.append(dist/(cosine_array[i].shape[0]-1))
    return array
avg_distance_bw_neighbours = get_avg_distance_bw_neighbours(cosine_array)
print(avg_distance_bw_neighbours)

[0.893363762103134, 0.9153651171798236, 0.8713317409295503, 0.9354169448180263, 0.9302272262520567]


In [12]:
def get_maxmin_distance_bw_neighbours(cosine_array):
    maxarray = []
    minarray = []
    for i in range(0,len(cosine_array)):
        j = 0
        maxn = 0
        minn = 1
        while j < cosine_array[i].shape[0]-1:
            maxn = maxn if maxn > cosine_array[i][j][j+1] else cosine_array[i][j][j+1]
            minn = minn if minn < cosine_array[i][j][j+1] else cosine_array[i][j][j+1]
            j+=1
        maxarray.append(maxn)
        minarray.append(minn)
    return maxarray,minarray
max_distance_bw_neighbours,min_distance_bw_neighbours = get_maxmin_distance_bw_neighbours(cosine_array)
print(max_distance_bw_neighbours)
print(min_distance_bw_neighbours)

[0.932874628083993, 0.9748138002490297, 0.9386361442643513, 0.9592303217827197, 0.9702919229616109]
[0.8299545314449256, 0.8421069373721458, 0.7972058816269103, 0.8778685590183576, 0.8750800365097381]


In [13]:
def get_avg_distance_bw_points(cosine_array):
    array = []
    for i in range(0,len(cosine_array)):
        dist = 0
        for j in range(0,cosine_array[i].shape[0]):
            for k in range(0,cosine_array[i].shape[1]):
                dist+=cosine_array[i][j][k]
        array.append(dist/(cosine_array[i].shape[0]*cosine_array[i].shape[1]))
    return array
avg_distance_bw_points = get_avg_distance_bw_points(cosine_array)
print(avg_distance_bw_points)

[0.3561959665437295, 0.3734389039368234, 0.3586218369196845, 0.37331717329228736, 0.3551577840854091]


In [14]:
def get_max_distance_bw_points(cosine_array):
    array = []
    for i in range(0,len(cosine_array)):
        maxn = 0
        for j in range(0,cosine_array[i].shape[0]):
            for k in range(0,cosine_array[i].shape[1]):
                if j!=k:
                    maxn = cosine_array[i][j][k] if maxn < cosine_array[i][j][k] else maxn
        array.append(maxn)
    return array
max_distance_bw_points = get_max_distance_bw_points(cosine_array)
print(max_distance_bw_points)

[0.932874628083993, 0.9748138002490297, 0.9386361442643513, 0.9592303217827197, 0.9702919229616109]


In [15]:

def get_clark_and_evans(cosine_array):
    array = []
    for i in range(0,len(cosine_array)):
        dist = 0
        for j in range(0,cosine_array[i].shape[0]):
            curr = 1
            if j-1>=0 and cosine_array[i][j-1][j] < curr :
                curr = cosine_array[i][j-1][j]
            if j+1<cosine_array[i].shape[0] and cosine_array[i][j][j+1] < curr:
                curr = cosine_array[i][j][j+1]
            dist+=curr
        array.append(2*dist/math.sqrt(cosine_array[i].shape[0]))
    return array
clark_and_evans = get_clark_and_evans(cosine_array)
print(clark_and_evans)

[9.29744926554458, 10.369937396938035, 8.199569648449204, 11.846259777137364, 11.156160630946264]


In [16]:
def get_avg_distance_to_nearest_neighbours(cosine_array):
    array = []
    for i in range(0,len(cosine_array)):
        dist = 0
        for j in range(0,cosine_array[i].shape[0]):
            curr = 1
            if j-1>=0 and cosine_array[i][j-1][j] < curr :
                curr = cosine_array[i][j-1][j]
            if j+1<cosine_array[i].shape[0] and cosine_array[i][j][j+1] < curr:
                curr = cosine_array[i][j][j+1]
            dist+=curr
        array.append(dist/cosine_array[i].shape[0])
    return array
avg_distance_to_nearest_neighbours = get_avg_distance_to_nearest_neighbours(cosine_array)
avg_distance_to_nearest_neighbours

[0.878526377995396,
 0.902587197233152,
 0.85486423038328,
 0.9250374768526111,
 0.9170307549717307]

In [17]:
def get_cumulative_frequency_distribution(cosine_array):
    array = []
    for i in range(0,len(cosine_array)):
        dist = 0
        for j in range(0,cosine_array[i].shape[0]):
            curr = 1
            if j-1>=0 and cosine_array[i][j-1][j] < curr :
                curr = cosine_array[i][j-1][j]
            if j+1<cosine_array[i].shape[0] and cosine_array[i][j][j+1] < curr :
                curr = cosine_array[i][j][j+1]
            if curr <= avg_distance_to_nearest_neighbours[i] :
                dist+=1
        array.append(dist/cosine_array[i].shape[0])
    return array
cumulative_frequency_distribution = get_cumulative_frequency_distribution(cosine_array)
cumulative_frequency_distribution

[0.5714285714285714,
 0.5151515151515151,
 0.5652173913043478,
 0.5121951219512195,
 0.40540540540540543]

In [18]:
def get_cumulative_frequency_distribution(cosine_array):
    array = []
    for i in range(0,len(cosine_array)):
        dist = 0
        for j in range(0,cosine_array[i].shape[0]):
            curr = 1
            if j-1>=0 and cosine_array[i][j-1][j] < curr :
                curr = cosine_array[i][j-1][j]
            if j+1<cosine_array[i].shape[0] and cosine_array[i][j][j+1] < curr :
                curr = cosine_array[i][j][j+1]
            if curr <= avg_distance_to_nearest_neighbours[i] :
                dist+=1
        array.append(dist/cosine_array[i].shape[0])
    return array
cumulative_frequency_distribution = get_cumulative_frequency_distribution(cosine_array)
cumulative_frequency_distribution

[0.5714285714285714,
 0.5151515151515151,
 0.5652173913043478,
 0.5121951219512195,
 0.40540540540540543]

In [19]:
euclidean_array = []
for i in range(0,len(array_of_strings)):
    tfidf_wm = tfidfvectorizer.fit_transform(array_of_strings[i])
    similarity_matrix = euclidean_distances(tfidf_wm)
    euclidean_array.append(similarity_matrix)

In [20]:
print(euclidean_array)

[array([[0.        , 0.55187271, 0.73190541, 0.87819587, 1.01261533,
        1.08603213, 1.12782953, 1.20012195, 1.22635485, 1.28332237,
        1.27703303, 1.29296485, 1.3066296 , 1.31378246, 1.3239307 ,
        1.30612346, 1.3174297 , 1.35093238, 1.37020247, 1.36887068,
        1.35395818, 1.36053163, 1.3613899 , 1.30395668, 1.27661477,
        1.17209181, 1.12573082, 1.11325649],
       [0.55187271, 0.        , 0.5250388 , 0.68397431, 0.86296105,
        0.9443071 , 0.99356819, 1.07385754, 1.11426877, 1.20520492,
        1.2686782 , 1.27643621, 1.2916772 , 1.29966277, 1.3110545 ,
        1.32281181, 1.33521483, 1.36150184, 1.38198125, 1.38101025,
        1.36338185, 1.36983577, 1.37054287, 1.35390759, 1.30795911,
        1.21823909, 1.16868609, 1.15829792],
       [0.73190541, 0.5250388 , 0.        , 0.42574111, 0.6757291 ,
        0.79462975, 0.84876095, 0.93964651, 0.97960008, 1.09232844,
        1.22926739, 1.27850874, 1.29422243, 1.3024231 , 1.31398168,
        1.32602143, 1.338

In [21]:
print(len(euclidean_array))

5


In [22]:
avg_distance_bw_neighbours_euclid = get_avg_distance_bw_neighbours(euclidean_array)
print(avg_distance_bw_neighbours_euclid)

[0.45814348755174794, 0.4068743235436039, 0.5009524897013377, 0.35615381555498654, 0.3678583228304632]


In [23]:
max_distance_bw_neighbours_euclid,min_distance_bw_neighbours_euclid = get_maxmin_distance_bw_neighbours(euclidean_array)
print(max_distance_bw_neighbours_euclid)
print(min_distance_bw_neighbours_euclid)

[0.5831731621998296, 0.561948507655024, 0.6368580978100062, 0.4942295842655359, 0.49983990134894596]
[0.366402434260492, 0.22443796359337362, 0.3503251510686857, 0.2855509699415512, 0.2437542903761464]


In [24]:
avg_distance_bw_points_euclid = get_avg_distance_bw_points(euclidean_array)
print(avg_distance_bw_points_euclid)

[1.0780297361407953, 1.0631080921252543, 1.0739067298626523, 1.0638192806327107, 1.0752204059103965]


In [25]:
centroid_array = []
for i in range(0,len(array_of_strings)):
    tfidf_wm = tfidfvectorizer.fit_transform(array_of_strings[i])
    temp = tfidf_wm[0]
    for j in range(1,tfidf_wm.shape[0]):
        temp+=tfidf_wm[j]
    centroid_array.append(temp/tfidf_wm.shape[0])



In [26]:
len(centroid_array)

5

In [27]:
euclidcentroid_array = []
for i in range(0,len(array_of_strings)):
    tfidf_wm = tfidfvectorizer.fit_transform(array_of_strings[i])
    similarity_matrix = euclidean_distances(tfidf_wm,centroid_array[i])
    euclidcentroid_array.append(similarity_matrix)
print(euclidcentroid_array)

[array([[0.88454169],
       [0.8527289 ],
       [0.82532897],
       [0.82390977],
       [0.80944724],
       [0.78269951],
       [0.7718543 ],
       [0.78167046],
       [0.7705205 ],
       [0.75994156],
       [0.75523465],
       [0.75895785],
       [0.77256854],
       [0.7721631 ],
       [0.75508868],
       [0.76480543],
       [0.7798835 ],
       [0.78632369],
       [0.7983275 ],
       [0.81548552],
       [0.79379995],
       [0.80589573],
       [0.82120946],
       [0.84271745],
       [0.82786258],
       [0.81569402],
       [0.83963559],
       [0.87627657]]), array([[0.87679906],
       [0.84037753],
       [0.8185479 ],
       [0.82379874],
       [0.82308004],
       [0.8186258 ],
       [0.78335117],
       [0.77221332],
       [0.75259838],
       [0.75326669],
       [0.72806538],
       [0.72980486],
       [0.74213868],
       [0.74095814],
       [0.74493051],
       [0.72700075],
       [0.77061988],
       [0.77203143],
       [0.75935536],
       [0.

In [28]:
print(len(euclidcentroid_array[0]))

28


In [29]:
print(cosine_array[0].shape)

(28, 28)


In [30]:
def get_avg_distance_bw_centroid_and_points(euclidcentroid_array):
    array = []
    for i in range(0,len(euclidcentroid_array)):
        dist = 0
        for j in range(0,len(euclidcentroid_array[i])):
            dist+=euclidcentroid_array[i][j]
        array.append(dist/len(euclidcentroid_array[i]))
    return array
avg_distance_bw_centroid_and_points = get_avg_distance_bw_centroid_and_points(euclidcentroid_array)
print(avg_distance_bw_centroid_and_points)

[array([0.80159188]), array([0.79023997]), array([0.79936786]), array([0.79004167]), array([0.80109667])]


In [31]:
def get_maxmin_distance_bw_centroid_and_points(euclidcentroid_array):
    maxarray = []
    minarray = []
    for i in range(0,len(euclidcentroid_array)):
        mx = 0
        mn = 1000
        for j in range(0,len(euclidcentroid_array[i])):
            if mx < euclidcentroid_array[i][j]:
                mx = euclidcentroid_array[i][j]
            if mn > euclidcentroid_array[i][j]:
                mn = euclidcentroid_array[i][j]
        maxarray.append(mx)
        minarray.append(mn)
    return maxarray,minarray
max_distance_bw_centroid_and_points,min_distance_bw_centroid_and_points = get_maxmin_distance_bw_centroid_and_points(euclidcentroid_array)
print(max_distance_bw_centroid_and_points)
print(min_distance_bw_centroid_and_points)

[array([0.88454169]), array([0.92623804]), array([0.91723553]), array([0.9199983]), array([0.94941452])]
[array([0.75508868]), array([0.72700075]), array([0.73814287]), array([0.72483534]), array([0.71589089])]


In [32]:
print(centroid_array[0])

  (0, 58)	0.00982730521738537
  (0, 1)	0.0338808593780755
  (0, 82)	0.04247817964361833
  (0, 87)	0.04247817964361833
  (0, 17)	0.04247817964361833
  (0, 73)	0.04247817964361833
  (0, 47)	0.04741251271265099
  (0, 4)	0.04741251271265099
  (0, 27)	0.05063307294732764
  (0, 71)	0.0495186803552858
  (0, 25)	0.0495186803552858
  (0, 53)	0.09842255990834192
  (0, 50)	0.052439416107603117
  (0, 80)	0.052439416107603117
  (0, 57)	0.054833658171613196
  (0, 48)	0.05676672390050393
  (0, 16)	0.05676672390050393
  (0, 89)	0.05418500903647992
  (0, 21)	0.057930894473847035
  (0, 59)	0.057930894473847035
  (0, 15)	0.14870304297933398
  (0, 88)	0.05353555336551381
  (0, 61)	0.05739258682170492
  (0, 12)	0.05053042091803313
  (0, 102)	0.05053042091803313
  :	:
  (0, 33)	0.05770368709586217
  (0, 83)	0.05770368709586217
  (0, 28)	0.05770368709586217
  (0, 77)	0.05770368709586217
  (0, 2)	0.054139943751726954
  (0, 5)	0.06240315841092577
  (0, 54)	0.057605052090071364
  (0, 91)	0.05432112879908292
  (

In [44]:
rows, cols = centroid_array[1].nonzero()

In [45]:
rows

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [46]:
cols.sort()

In [47]:
cols

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120], dtype=int32)